# Import Contour Data

In [1]:
import numpy as np
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
CS=Dataset('META3.1exp_DT_allsat_Cyclonic_short_19930101_20200307.nc')

In [3]:
CL=Dataset('META3.1exp_DT_allsat_Cyclonic_long_19930101_20200307.nc')

In [4]:
print(CS.variables.keys())

dict_keys(['amplitude', 'cost_association', 'effective_area', 'effective_contour_height', 'effective_contour_latitude', 'effective_contour_longitude', 'effective_contour_shape_error', 'effective_radius', 'inner_contour_height', 'latitude', 'latitude_max', 'longitude', 'longitude_max', 'num_contours', 'num_point_e', 'num_point_s', 'observation_flag', 'observation_number', 'speed_area', 'speed_average', 'speed_contour_height', 'speed_contour_latitude', 'speed_contour_longitude', 'speed_contour_shape_error', 'speed_radius', 'time', 'track', 'uavg_profile'])


# Import ADT Data

In [5]:
SLA=Dataset('../copernicus/cmems_obs-sl_glo_phy-ssh_my_allsat-l4-duacs-0.25deg_P1D_1704119807053.nc')

In [6]:
print(SLA.variables.keys())

dict_keys(['adt', 'crs', 'latitude', 'time', 'longitude'])


In [7]:
adt=SLA.variables['adt'][:]

In [8]:
adt.shape

(3653, 81, 181)

In [9]:
maplat=SLA.variables['latitude'][:]
maplon=SLA.variables['longitude'][:]

In [10]:
maplat

masked_array(data=[24.875, 25.125, 25.375, 25.625, 25.875, 26.125, 26.375,
                   26.625, 26.875, 27.125, 27.375, 27.625, 27.875, 28.125,
                   28.375, 28.625, 28.875, 29.125, 29.375, 29.625, 29.875,
                   30.125, 30.375, 30.625, 30.875, 31.125, 31.375, 31.625,
                   31.875, 32.125, 32.375, 32.625, 32.875, 33.125, 33.375,
                   33.625, 33.875, 34.125, 34.375, 34.625, 34.875, 35.125,
                   35.375, 35.625, 35.875, 36.125, 36.375, 36.625, 36.875,
                   37.125, 37.375, 37.625, 37.875, 38.125, 38.375, 38.625,
                   38.875, 39.125, 39.375, 39.625, 39.875, 40.125, 40.375,
                   40.625, 40.875, 41.125, 41.375, 41.625, 41.875, 42.125,
                   42.375, 42.625, 42.875, 43.125, 43.375, 43.625, 43.875,
                   44.125, 44.375, 44.625, 44.875],
             mask=False,
       fill_value=1e+20,
            dtype=float32)

In [11]:
maplon

masked_array(data=[134.875, 135.125, 135.375, 135.625, 135.875, 136.125,
                   136.375, 136.625, 136.875, 137.125, 137.375, 137.625,
                   137.875, 138.125, 138.375, 138.625, 138.875, 139.125,
                   139.375, 139.625, 139.875, 140.125, 140.375, 140.625,
                   140.875, 141.125, 141.375, 141.625, 141.875, 142.125,
                   142.375, 142.625, 142.875, 143.125, 143.375, 143.625,
                   143.875, 144.125, 144.375, 144.625, 144.875, 145.125,
                   145.375, 145.625, 145.875, 146.125, 146.375, 146.625,
                   146.875, 147.125, 147.375, 147.625, 147.875, 148.125,
                   148.375, 148.625, 148.875, 149.125, 149.375, 149.625,
                   149.875, 150.125, 150.375, 150.625, 150.875, 151.125,
                   151.375, 151.625, 151.875, 152.125, 152.375, 152.625,
                   152.875, 153.125, 153.375, 153.625, 153.875, 154.125,
                   154.375, 154.625, 154.875, 155.1

# Calculate Time Range

In [12]:
time=CS.variables['time'][:]

In [13]:
import datetime

In [14]:
datetime.datetime.strptime('1950-01-01','%Y-%m-%d')+datetime.timedelta(days=21550)

datetime.datetime(2009, 1, 1, 0, 0)

In [15]:
datetime.datetime.strptime('2010-01-01','%Y-%m-%d')-datetime.datetime.strptime('1950-01-01','%Y-%m-%d')

datetime.timedelta(days=21915)

Time from 2009-1-1 to 2009-12-31 should be 21550 to 21914.
Time from 2010-1-1 to 2020-1-1 should be 21915 to 25567.

# Data Transform

## Get Contour Coordinate within Northwest Pacific Area

In [16]:
time_cs=CS.variables['time'][:]
center_lon_cs=CS.variables['longitude'][:]
center_lat_cs=CS.variables['latitude'][:]

In [17]:
time_cl=CL.variables['time'][:]
center_lon_cl=CL.variables['longitude'][:]
center_lat_cl=CL.variables['latitude'][:]

In [18]:
import time as tm

保存为：

contour_coordinate:顶点的坐标，格式为[[50个lon],[50个lat]],即list中每个元素shape均为(2,50)

contour_time:顶点坐标获取的时间

contour_index:顶点坐标在原始数据中对应的序号

## Transform Coordinate into Grid

In [19]:
contour_coordinate_cs=np.load('contour_coordinate_cs.npy')
contour_time_cs=np.load('contour_time_cs.npy')
contour_index_cs=np.load('contour_index_cs.npy')

In [20]:
contour_coordinate_cl=np.load('contour_coordinate_cl.npy')
contour_time_cl=np.load('contour_time_cl.npy')
contour_index_cl=np.load('contour_index_cl.npy')

In [21]:
def point_in_polygon(x, y, poly_x, poly_y):    #判断grid坐标是否位于涡旋边界内部，输入grid lon, grid lat, contour lon, contour lat
    num_vertices = len(poly_x)
    inside = False
    j = num_vertices - 1
    for i in range(num_vertices):
        if ((poly_y[i] > y) != (poly_y[j] > y)) and (x < (poly_x[j] - poly_x[i]) * (y - poly_y[i]) / (poly_y[j] - poly_y[i]) + poly_x[i]):
            inside = not inside
        j = i
    return inside

改了个时间

In [22]:
grid_total=[]
start_time=tm.time()

for time in range(21550,21915):
    grid_data = np.zeros((len(maplat), len(maplon)))
    for k in range(len(contour_time_cs)):
        if contour_time_cs[k]==time:
            #print(k,time)
            for i in range(len(maplat)):
                for j in range(len(maplon)):
                    if point_in_polygon(maplon[j], maplat[i], contour_coordinate_cs[k][0], contour_coordinate_cs[k][1]):
                        grid_data[i, j] = -1
    
    grid_total.append(grid_data)
    end_time=tm.time()
    print(str(time)+', '+str(end_time-start_time)+'s')

21550, 2.5317342281341553s
21551, 5.894493818283081s
21552, 10.207346677780151s
21553, 16.042335510253906s
21554, 23.499160051345825s
21555, 28.490352392196655s
21556, 31.02086877822876s
21557, 33.54995942115784s
21558, 35.25698113441467s
21559, 37.779173135757446s
21560, 39.48285436630249s
21561, 40.363147258758545s
21562, 41.242427349090576s
21563, 43.76742458343506s
21564, 47.184775829315186s
21565, 49.70930814743042s
21566, 51.3997118473053s
21567, 54.757078647613525s
21568, 60.578742265701294s
21569, 65.567138671875s
21570, 71.3843286037445s
21571, 79.6661422252655s
21572, 90.46529841423035s
21573, 99.57603788375854s
21574, 107.8959150314331s
21575, 115.3799352645874s
21576, 122.84308457374573s
21577, 131.98422598838806s
21578, 139.44559717178345s
21579, 144.4585075378418s
21580, 149.4479124546051s
21581, 156.07905292510986s
21582, 161.06395149230957s
21583, 166.06052827835083s
21584, 171.06815481185913s
21585, 177.6959993839264s
21586, 182.6876516342163s
21587, 187.66653609275818

21863, 1196.8121135234833s
21864, 1200.1803646087646s
21865, 1202.7138171195984s
21866, 1205.2362761497498s
21867, 1207.7806231975555s
21868, 1211.147213935852s
21869, 1212.858644247055s
21870, 1214.573165178299s
21871, 1218.7610504627228s
21872, 1222.126796245575s
21873, 1225.4971787929535s
21874, 1229.698795080185s
21875, 1233.8867328166962s
21876, 1237.2658233642578s
21877, 1241.4489395618439s
21878, 1245.6346085071564s
21879, 1248.9899504184723s
21880, 1251.5340762138367s
21881, 1254.9033377170563s
21882, 1257.4493329524994s
21883, 1259.9685127735138s
21884, 1261.6824462413788s
21885, 1263.394969701767s
21886, 1265.0957114696503s
21887, 1267.632513999939s
21888, 1271.0005850791931s
21889, 1275.153811454773s
21890, 1278.5111198425293s
21891, 1280.2180635929108s
21892, 1281.9175171852112s
21893, 1284.450223684311s
21894, 1286.1596479415894s
21895, 1287.8663084506989s
21896, 1289.578374862671s
21897, 1292.109438419342s
21898, 1293.8219063282013s
21899, 1295.531466960907s
21900, 1298.0

In [23]:
grid_total=np.array(grid_total)
np.save('grid_cs_test',grid_total)

In [24]:
grid_total=[]
start_time=tm.time()
for time in range(21550,21915):
    grid_data = np.zeros((len(maplat), len(maplon)))
    for k in range(len(contour_time_cl)):
        if contour_time_cl[k]==time:
            #print(k,time)
            for i in range(len(maplat)):
                for j in range(len(maplon)):
                    if point_in_polygon(maplon[j], maplat[i], contour_coordinate_cl[k][0], contour_coordinate_cl[k][1]):
                        grid_data[i, j] = -1
    
    grid_total.append(grid_data)
    end_time=tm.time()
    print(str(time)+', '+str(end_time-start_time)+'s')

21550, 70.93159818649292s
21551, 141.1001706123352s
21552, 209.7184829711914s
21553, 278.3438696861267s
21554, 349.3301742076874s
21555, 418.72853779792786s
21556, 488.9442546367645s
21557, 560.8579716682434s
21558, 635.288697719574s
21559, 705.4592716693878s
21560, 776.5475578308105s
21561, 848.5276999473572s
21562, 921.8151607513428s
21563, 992.7733638286591s
21564, 1065.4711241722107s
21565, 1139.013588666916s
21566, 1208.272539615631s
21567, 1277.5991933345795s
21568, 1348.5825414657593s
21569, 1420.5304872989655s
21570, 1494.7210912704468s
21571, 1569.119291305542s
21572, 1642.8083634376526s
21573, 1714.6359016895294s
21574, 1789.004323720932s
21575, 1863.4235064983368s
21576, 1940.2862923145294s
21577, 2017.1618840694427s
21578, 2094.7918260097504s
21579, 2173.2651312351227s
21580, 2250.1085011959076s
21581, 2328.435288667679s
21582, 2405.211772441864s
21583, 2482.0970137119293s
21584, 2562.0721032619476s
21585, 2641.1676619052887s
21586, 2720.492842197418s
21587, 2798.8455328941

21860, 21928.167801618576s
21861, 21994.69642853737s
21862, 22060.36884355545s
21863, 22126.5832157135s
21864, 22191.293343305588s
21865, 22256.98757314682s
21866, 22321.639798641205s
21867, 22385.848355054855s
21868, 22448.82445168495s
21869, 22511.164650440216s
21870, 22572.62598991394s
21871, 22630.886864185333s
21872, 22687.452177286148s
21873, 22745.62686395645s
21874, 22805.533646821976s
21875, 22867.844455957413s
21876, 22930.1760802269s
21877, 22995.77047419548s
21878, 23060.510560274124s
21879, 23126.129298210144s
21880, 23191.95803141594s
21881, 23257.557334423065s
21882, 23323.536650896072s
21883, 23391.742721796036s
21884, 23461.610910654068s
21885, 23531.196046352386s
21886, 23604.46378517151s
21887, 23677.579380512238s
21888, 23748.17465376854s
21889, 23816.681468963623s
21890, 23885.350445747375s
21891, 23953.175619363785s
21892, 24022.63312935829s
21893, 24091.165529727936s
21894, 24159.88344359398s
21895, 24230.979696273804s
21896, 24299.407894849777s
21897, 24369.6740

In [25]:
grid_total=np.array(grid_total)
np.save('grid_cl_test',grid_total)